In [1]:
import numpy as np
np.set_printoptions(linewidth=200, precision=4) # to print wide matrices

import simphony
simphony.Config.set_platform('gpu')
simphony.Config.set_matplotlib_format('retina')

from matplotlib import pyplot as plt

In [2]:
plt.rcParams.update({'font.size': 14})